In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

pd.set_option('display.max_columns', None)


DATA = '/Users/Marcy_Student/Desktop/marcy/DA2025_Lectures_Kevin/Mod6/Mod6_Project/Data/Motor_Vehicle_Collisions_-_Crashes_20251209.csv'

df = pd.read_csv(DATA)

# Drop bad coordinates
df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['LONGITUDE'], df['LATITUDE']),
    crs='EPSG:4326'
)

# Load NYC borough shapes
boros = gpd.read_file('/Users/Marcy_Student/Desktop/marcy/DA2025_Lectures_Kevin/Mod6/Mod6_Project/nybb_25d/nybb.shp')
boros = boros.to_crs('EPSG:4326')

# Spatial join: assign borough to each crash point
df = gpd.sjoin(gdf, boros[['BoroName', 'geometry']], how='left', predicate='within')

df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'],format="%d/%m/%Y", errors='coerce')
timestamp = pd.to_datetime(df['CRASH TIME'], format='%H:%M', errors='coerce')
df['CRASH TIME'] = timestamp.dt.strftime('%H:%M')
nan_counts_per_column = df.isnull().sum()

df

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,geometry,index_right,BoroName
1,2022-02-01,12:50,QUEENS,11373.0,40.735065,-73.881840,"(40.735065, -73.88184)",GRAND AVENUE,VANKLEECK STREET,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,4492016,Sedan,Bike,NaN,NaN,NaN,POINT (-73.88184 40.73506),2.0,Queens
2,2022-02-01,00:30,NaN,NaN,40.699190,-73.914690,"(40.69919, -73.91469)",IRVING AVENUE,NaN,NaN,0,0,0,0,0,0,0,0,Turning Improperly,NaN,NaN,NaN,NaN,4491657,Sedan,NaN,NaN,NaN,NaN,POINT (-73.91469 40.69919),4.0,Brooklyn
3,2022-02-01,08:44,BRONX,10460.0,40.842224,-73.887760,"(40.842224, -73.88776)",NaN,NaN,1902 MARMION AVENUE,0,0,0,0,0,0,0,0,Driver Inexperience,Unspecified,NaN,NaN,NaN,4491839,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,POINT (-73.88776 40.84222),1.0,Bronx
4,2022-02-01,04:15,NaN,NaN,40.743210,-73.896080,"(40.74321, -73.89608)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4491367,Sedan,NaN,NaN,NaN,NaN,POINT (-73.89608 40.74321),2.0,Queens
6,2022-02-01,03:00,NaN,NaN,40.762940,-73.860670,"(40.76294, -73.86067)",GRAND CENTRAL PKWY,NaN,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4491464,Ambulance,NaN,NaN,NaN,NaN,POINT (-73.86067 40.76294),2.0,Queens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370554,2025-05-12,09:01,QUEENS,11377.0,40.752064,-73.900870,"(40.752064, -73.90087)",61 ST,34 AVE,NaN,1,0,1,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4862309,Sedan,NaN,NaN,NaN,NaN,POINT (-73.90087 40.75206),2.0,Queens
370555,2025-05-12,08:20,BROOKLYN,11221.0,40.687120,-73.920975,"(40.68712, -73.920975)",NaN,NaN,44 HOWARD AVE,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4862459,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,POINT (-73.92098 40.68712),4.0,Brooklyn
370557,2025-05-12,12:24,BROOKLYN,11221.0,40.688790,-73.916900,"(40.68879, -73.9169)",BUSHWICK AVE,CORNELIA ST,NaN,3,0,0,0,0,0,3,0,Fatigued/Drowsy,NaN,NaN,NaN,NaN,4862592,Sedan,NaN,NaN,NaN,NaN,POINT (-73.9169 40.68879),4.0,Brooklyn
370558,2025-05-12,13:47,BRONX,10475.0,40.868770,-73.831730,"(40.86877, -73.83173)",BARTOW AVE,BAYCHESTER AVE,NaN,2,0,0,0,0,0,2,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4862768,Bus,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,POINT (-73.83173 40.86877),1.0,Bronx


In [7]:
df['BoroName'].isna().sum()

np.int64(5557)